In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.stem.porter import PorterStemmer
from string import punctuation
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')


import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/omarelshobky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/omarelshobky/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
train = pd.read_csv("Dataset/train.csv" ,encoding='unicode_escape')
train.head(5)

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


now i will drop country and population ect... because they are irrelevant to the problem, also dropping Age of user because this feature is not supported in the other dataset and also not that relevant to the problem.

In [3]:
train.drop(axis = 1,labels = ['Population -2020','Land Area (Km²)','Country','Age of User',"Density (P/Km²)",'text'],inplace=True)
train = train[train.selected_text.notnull()]
train.reset_index(drop=True,inplace = True )

train['Time of Tweet'][train['Time of Tweet'] == 'morning'] = 0
train['Time of Tweet'][train['Time of Tweet'] == 'night'] = 1
train['Time of Tweet'][train['Time of Tweet'] == 'noon'] = 2


train['sentiment'][train['sentiment'] == 'negative'] = 0
train['sentiment'][train['sentiment'] == 'neutral'] = 1
train['sentiment'][train['sentiment'] == 'positive'] = 2
train['sentiment'] = (train['sentiment'].astype('int'))

train.head(5)

,textID,selected_text,sentiment,Time of Tweet
0,cb774db0d1,"I`d have responded, if I were going",1,0
1,549e992a42,Sooo SAD,0,2
2,088c60f138,bullying me,0,1
3,9642c003ef,leave me alone,0,0
4,358bd9e861,"Sons of ****,",0,2


In [4]:
train2 = pd.read_csv("Dataset/training.1600000.processed.noemoticon.csv",delimiter=',', encoding='ISO-8859-1')
train2.head(5)

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


we will keep only the columns in the previous dataset and cahnge the column names to match

In [5]:
train2 = train2[train2[train2.columns[-1]].notnull()]
train2['sentiment'] = train2['polarity of tweet\xa0']//2
train2['textID'] = train2[train2.columns[1]]
train2['Time of Tweet'] = train2[train2.columns[2]]
train2.drop(labels = [train2.columns[0],train2.columns[2],train2.columns[3],train2.columns[4]],axis=1,inplace=True)

train2['selected_text'] = train2[train2.columns[1]]
train2 = train2[['textID','selected_text','sentiment','Time of Tweet']]
train2.head(5)

,textID,selected_text,sentiment,Time of Tweet
0,1467810672,is upset that he can't update his Facebook by ...,0,Mon Apr 06 22:19:49 PDT 2009
1,1467810917,@Kenichan I dived many times for the ball. Man...,0,Mon Apr 06 22:19:53 PDT 2009
2,1467811184,my whole body feels itchy and like its on fire,0,Mon Apr 06 22:19:57 PDT 2009
3,1467811193,"@nationwideclass no, it's not behaving at all....",0,Mon Apr 06 22:19:57 PDT 2009
4,1467811372,@Kwesidei not the whole crew,0,Mon Apr 06 22:20:00 PDT 2009


In [6]:
def categ_time(arr_of_dates):
    arr_of_time = np.zeros(len(arr_of_dates), dtype = int)
    i=0
    for d in arr_of_dates:
        t = datetime.strptime(d, '%a %b %d %H:%M:%S PDT %Y')
        t =  t.hour
        if (t >= 0 and t < 8):
            t = 1  #night
        elif(t >= 8 and t<= 16):
            t = 0 #morning
        else:
            t= 2
        arr_of_time[i] += t
        i += 1
    return arr_of_time

In [7]:
train2['Time of Tweet'] = categ_time(train2['Time of Tweet'][:].values)
train2.head(5)

,textID,selected_text,sentiment,Time of Tweet
0,1467810672,is upset that he can't update his Facebook by ...,0,2
1,1467810917,@Kenichan I dived many times for the ball. Man...,0,2
2,1467811184,my whole body feels itchy and like its on fire,0,2
3,1467811193,"@nationwideclass no, it's not behaving at all....",0,2
4,1467811372,@Kwesidei not the whole crew,0,2


In [8]:
train2['sentiment'].value_counts()

0    799996
2    248576
Name: sentiment, dtype: int64

In [9]:
train['sentiment'].value_counts()

1    11117
2     8582
0     7781
Name: sentiment, dtype: int64

there is a huge bias in the classes.

In [10]:
neg = train[train['sentiment'] == 0]
neu = train[train['sentiment'] == 1]
pos = train[train['sentiment'] == 2]

In [11]:
neg2 = train2[train2['sentiment'] == 0]
pos2 = train2[train2['sentiment'] == 2]

In [12]:
pos = pos.append(pos2)
neg = neg.append(neg2)
neg = neg.sample(replace=False,n=len(pos),random_state=0)

In [13]:
train = neu.append(neg)
train = train.append(pos)
train['sentiment'].value_counts()

0    257158
2    257158
1     11117
Name: sentiment, dtype: int64

In [14]:
train = train.sample(frac = 1).reset_index(drop=True)
train

,textID,selected_text,sentiment,Time of Tweet
0,1564782520,"my internet is back again, yipee!",2,1
1,1691418675,@Vyingbymarge LOL I always try to have time f...,2,2
2,1883005845,im kinda mad and dont understand this,0,1
3,2011542373,Just what I need hanging out with my friends t...,0,2
4,2055613874,"@LindaJosey yeah, but it started at 3 am yeste...",0,0
...,...,...,...,...
525428,1836435746,@keitharichall I called the guy back and told ...,2,1
525429,1559452324,"making pork, mashed potatoes and veg for dinne...",2,0
525430,1573836243,bk 4rom scl my english teacher cryied coz she ...,0,1
525431,2196844052,"@cortneyofeden Yup, currently mirroring that t...",0,0


In [15]:
train['Time of Tweet'].value_counts()

1    237718
2    150874
0    136841
Name: Time of Tweet, dtype: int64

this bias is acceptable. now we need to preprocess the text now.

In [16]:
def preprocess_text(tweets):
    stemmer = PorterStemmer()
    text = [] # array of new tweets
    swears = [] # no of swears per tweet
    bag_of_words = set({})
    for i in range(0,len(tweets)):
        if(i%10000 == 0):
            print(f'iteration :{i}')
        s = 0 # swear counter
        tweet = re.sub('[^a-zA-Z*@\s\n]', '', tweets[i]) # removes any none alphabetic letter + allow swears and usernames to be removed as a whole later
        tweet = re.sub(r'((\w)\2{2,})', '', tweet) # to remove words like aaaaah zzzzz etc.
        tweet = tweet.lower()
        tweet = tweet.split()
        val = nltk.pos_tag(tweet) # gets word type
        tweet = []
        for j in range(0,len(val)-1):
            val2 = val[j]
            stemmed_word = stemmer.stem(val2[0])
            if(len(val2[0]) <= 1):
                continue
            if("*" in stemmed_word): 
                tweet.append('*')
                bag_of_words.add('*')
                s += 1
            elif(val2[1] in ['NN','NNS','NNPS','NNP','VBZ','VBP','VBN','VBG','VBD','VB','RBS', 'RB', 'RBR','JJ','JJR','JJS']): # allowing verbs nouns adjctives and adverbs only
                if (not (('http' in stemmed_word) or ('@' in stemmed_word) or ('www' in stemmed_word))): # and also word has to be not a link or a username
                    tweet.append(stemmed_word)
                    bag_of_words.add(stemmed_word)
        
        swears.append(s)   

        tweet = [word for word in tweet]
        text.append(' '.join(tweet))
    
    return text,bag_of_words,swears

In [19]:
def FE(train,text_col_name = 'selected_text' , max_feat = 1500, ngram = (1,1)):
    labels = train['sentiment'].values
    msgs1,bag,swears = preprocess_text(train[text_col_name])

    tfidf_v = TfidfVectorizer(max_features=max_feat,ngram_range=ngram)
    Feat_vec = tfidf_v.fit_transform(msgs1).toarray()
    cols = tfidf_v.get_feature_names_out()
    print("After vectorization")
    final = pd.DataFrame(Feat_vec,columns = cols)
    Feat_vec = np.column_stack([Feat_vec,train['Time of Tweet'][:].values])
    
    print("after adding")                    
    final['Time of Tweet'] = train['Time of Tweet'][:]
    final['number_of_swears'] = np.array(swears,dtype=float)
    
    return final,Feat_vec,labels,bag,max(swears),tfidf_v

In [20]:
df,features,labels,bag,max_swears,tfidf_v = FE(train)

iteration :0
iteration :10000
iteration :20000
iteration :30000
iteration :40000
iteration :50000
iteration :60000
iteration :70000
iteration :80000
iteration :90000
iteration :100000
iteration :110000
iteration :120000
iteration :130000
iteration :140000
iteration :150000
iteration :160000
iteration :170000
iteration :180000
iteration :190000
iteration :200000
iteration :210000
iteration :220000
iteration :230000
iteration :240000
iteration :250000
iteration :260000
iteration :270000
iteration :280000
iteration :290000
iteration :300000
iteration :310000
iteration :320000
iteration :330000
iteration :340000
iteration :350000
iteration :360000
iteration :370000
iteration :380000
iteration :390000
iteration :400000
iteration :410000
iteration :420000
iteration :430000
iteration :440000
iteration :450000
iteration :460000
iteration :470000
iteration :480000
iteration :490000
iteration :500000
iteration :510000
iteration :520000
After vectorization
after adding


In [21]:
df.head(5)

,abl,about,absolut,account,ach,act,actual,ad,adam,add,...,yo,youll,your,youtub,youv,yr,yummi,yup,Time of Tweet,number_of_swears
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [22]:
"""split 0.2"""
split = int(0.3*(len(features)))
X_train = features[:split+1]
y_train = labels[:split+1]

In [23]:
X_test = features[split+1:]
y_test = labels[split+1:]

In [32]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [33]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(penalty = 'l2', solver = 'saga', C = 100, random_state=0)

model.fit(X_train,y_train)

X_train_prediction = model.predict(X_test)
training_data_accuracy = metrics.accuracy_score(X_train_prediction, y_test)
print('Accuracy score of the training data:',training_data_accuracy)

Accuracy score of the training data: 0.7242953428873609


In [35]:
test1 = pd.read_csv("Dataset/test.csv",encoding= 'unicode_escape')

test1.drop(axis = 1,labels = ['Population -2020','Land Area (Km²)','Country','Age of User',"Density (P/Km²)"],inplace=True)
test1 = test1[test1.text.notnull()]
test1.reset_index(drop=True,inplace = True )

test1['Time of Tweet'][test1['Time of Tweet'] == 'morning'] = 0 
test1['Time of Tweet'][test1['Time of Tweet'] == 'night'] = 1
test1['Time of Tweet'][test1['Time of Tweet'] == 'noon'] = 2


test1['sentiment'][test1['sentiment'] == 'negative'] = 0
test1['sentiment'][test1['sentiment'] == 'neutral'] = 1
test1['sentiment'][test1['sentiment'] == 'positive'] = 2

test1.head(5)

,textID,text,sentiment,Time of Tweet
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,1,0
1,96d74cb729,Shanghai is also really exciting (precisely -...,2,2
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",0,1
3,01082688c6,happy bday!,2,0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,2,2


In [36]:
yt = test1['sentiment'].values
idx = test1['textID'].values

#features
xt = tfidf_v.transform(test1['text']).todense()
xt = np.column_stack([xt,test1['Time of Tweet']])

#create df just for analysis
tfidf_df_test = pd.DataFrame(tfidf_v.transform(test1['text']).todense())
tfidf_df_test.columns = sorted(tfidf_v.get_feature_names_out())
tfidf_df_test['Time of Tweet'] = test1['Time of Tweet']


ytp = model.predict(tfidf_df_test)

submission = pd.DataFrame(tfidf_v.transform(test1['text']).todense(),index = idx)
submission.columns = ['labels']
submission.to_csv('Dataset/submit.csv')

NameError: name 'tfidf_v' is not defined

In [19]:
df.head(5)

,aa,aah,aaron,ab,abandon,abbi,abc,abil,abit,abl,...,yy,zac,zack,zealand,zoe,zombi,zone,zoo,Time of Tweet,number_of_swears
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0


In [21]:
df.shape

(525433, 5002)

In [23]:
features.shape

(525433, 5002)

more analysis on swears and sentiment

In [24]:
df['number_of_swears'][labels==0].value_counts()

0.0     254986
2.0       1796
4.0        350
6.0         23
8.0          2
10.0         1
Name: number_of_swears, dtype: int64

In [25]:
df['number_of_swears'][labels==1].value_counts()

0.0    10846
2.0      239
4.0       28
6.0        3
8.0        1
Name: number_of_swears, dtype: int64

In [26]:
df['number_of_swears'][labels==2].value_counts()

0.0     254965
2.0       1746
4.0        424
6.0         17
8.0          5
10.0         1
Name: number_of_swears, dtype: int64

In [27]:
class NeuralNet(nn.Module):
    def __init__(self,inp,hidden,op):
        super(NeuralNet,self).__init__()
        self.l1 = nn.Linear(inp,(inp*9)//10)
        self.l2 = nn.Linear((inp*9)//10,(inp*7)//10)
        self.l3 = nn.Linear((inp*7)//10,(inp*5)//10)
        self.l4 = nn.Linear((inp*5)//10,hidden)
        self.l5 = nn.Linear(hidden,hidden//2)
        self.l6 = nn.Linear(hidden//2,op)
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.tanh(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        out = self.tanh(out)
        out = self.l5(out)
        out = self.relu(out)
        out = self.l6(out)
        return out

In [28]:
class data(Dataset):
    def __init__(self,x,y): # x and y are np arrays with the values of the merged df
        self.n_samples = len(y)
        self.x_data = x
        self.y_data = y
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return self.n_samples

In [29]:
def train_nn(x_train,y_train,batch,hidden,alpha=0.001,num_epochs=100):
    dataset = data(x_train,y_train)
    loader = DataLoader(dataset = dataset , batch_size = batch, shuffle = True, num_workers = 0)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = NeuralNet(dataset.shape[1],hidden,3).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = alpha)
    for i in range(num_epochs):
        for words,labels in loader:
            words.to(device)
            labels.to(device)
            outputs = model(words)
            loss = criterion(outputs,labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if ((i+1) % 20 == 0):
            print(f"epoch {i+1}/{num_epochs} , loss = {loss.item():.4f}")
    print(f"epoch {i+1}/{num_epochs} , final loss = {loss.item():.4f}")
    return model

In [33]:
"""split 0.2"""
split = int(0.3*(len(features)))
X_train = features[:split+1]
y_train = labels[:split+1]

In [34]:
X_test = features[split+1:]
y_test = labels[split+1:]

In [35]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(penalty = 'l2',solver='saga',C = 100, random_state=0)

model.fit(X_train,y_train)

X_train_prediction = model.predict(X_test)
training_data_accuracy = metrics.accuracy_score(X_train_prediction, y_test)
print('Accuracy score of the training data:',training_data_accuracy)

Accuracy score of the training data: 0.7244394417663804


In [4]:
test1 = pd.read_csv("Dataset/test.csv",encoding= 'unicode_escape')
test1.head(5)

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0


In [11]:
test1 = pd.read_csv("Dataset/test.csv",encoding= 'unicode_escape')

test1.drop(axis = 1,labels = ['Population -2020','Land Area (Km²)','Country','Age of User',"Density (P/Km²)"],inplace=True)
test1 = test1[test1.text.notnull()]
test1.reset_index(drop=True,inplace = True )

test1['Time of Tweet'][test1['Time of Tweet'] == 'morning'] = 0 
test1['Time of Tweet'][test1['Time of Tweet'] == 'night'] = 1
test1['Time of Tweet'][test1['Time of Tweet'] == 'noon'] = 2


test1['sentiment'][test1['sentiment'] == 'negative'] = 0
test1['sentiment'][test1['sentiment'] == 'neutral'] = 1
test1['sentiment'][test1['sentiment'] == 'positive'] = 2

test1.head(5)

,textID,text,sentiment,Time of Tweet
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,1,0
1,96d74cb729,Shanghai is also really exciting (precisely -...,2,2
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",0,1
3,01082688c6,happy bday!,2,0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,2,2


In [ ]:
yt = test1['sentiment'].values
idx = test1['textID'].values

#features
xt = tfidf_v.transform(test1['text']).todense()
xt = np.column_stack([xt,test1['Time of Tweet']])

#create df just for analysis
tfidf_df_test = pd.DataFrame(tfidf_v.transform(test1['text']).todense())
tfidf_df_test.columns = sorted(tfidf_v.get_feature_names_out())
tfidf_df_test['Time of Tweet'] = test1['Time of Tweet']


ytp = model.predict(tfidf_df_test)

submission = pd.DataFrame(tfidf_v.transform(test1['text']).todense(),index = idx)
submission.columns = ['labels']
submission.to_csv('Dataset/submit.csv')